In [1]:
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search,Q
from credentials import usr_pwd

In [2]:
hosts = ["10.1.8.6:9200"]
user,pas = usr_pwd()

In [3]:
es = Elasticsearch(hosts,
                  timeout = 6000,
                  use_ssl = True,
                  verify_certs = False,
                  ssl_show_warn = False,
                  raise_on_error = False,
                  http_auth = (user,pas)
                  )

In [4]:
es.ping()

True

In [5]:
import pandas as pd
import numpy as np

/home/pavan_rs/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [6]:
pwd

'/mnt/cantire-analytics/datarepo/pavan rs/elasticsearch_works'

In [9]:
cd zz

/mnt/cantire-analytics/datarepo/zz


In [27]:
df = pd.read_excel("entityExtractionOA_sep_oct.xlsx")
print(df.shape)
df.head(1)

(1774, 10)


,vendor_number,vendor_name,cbm_number,cbm_name,contract_number,contract_date,amount,program_name,filename,md5
0,9680,EURO-PRO CORPORATION,30.0,NaN,NaN,2022-09-12 00:00:00,200.06,Vendor Funds to Support Webshop Orders for DWO...,Shark_9680_-_Margin_O&A_Q1_Q2_-_Webshop_Orders...,dabd09db079b69c5e71cff3fd6ca995c


In [28]:
len(df["md5"].unique())

1602

In [29]:
md5s = df["md5"].unique()

In [30]:
md5s[0]

'dabd09db079b69c5e71cff3fd6ca995c'

In [31]:
import csv
header = ["md5","from","to","subject","email_date"]
with open("entityExtrOA__.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [55]:
# body = {
#         "query":{
#             "bool":{
#                 "must":[
#                 {
#                     "match_phrase_prefix":{
#                         "fileType":"Email"
#                     }
#                 },
#                 {
#                     "match_phrase_prefix":{
#                         "Attachment.MD5":"0623d50b518c76f9573b63a5c6d9ac3c"
#                     }
#                 }]
#             }
#         }
#     }
# res = es.search(index="cantire-2022-*",body=body)
# res["hits"]["hits"]

In [32]:
from tqdm import tqdm
i=0
lis = []
for name in tqdm(md5s):
    i+=1
    body = {
        "query":{
            "bool":{
                "must":[{
                    "match_phrase_prefix":{
                        "Attachment.MD5":name
                    }
                },
                {
                    "match_phrase_prefix":{
                        "fileType":"Email"
                    }
                }]
            }
        }
    }
    res = es.search(index="cantire-2022-*",body = body)
    frm = []
    to = []
    sub = []
    date = []
    md5 = []
    try:
        f = res["hits"]["hits"][0]["_source"]["metaData"]["from"]
        if type(f)!=str:
            frm.extend(f)
        else:
            frm.append(f)
    except:
        pass
    try:
        t = res["hits"]["hits"][0]["_source"]["metaData"]["to"]
        if type(t)!=str:
            to.extend(t)
        else:
            to.append(t)
    except:
        pass
    try:
        s = res["hits"]["hits"][0]["_source"]["metaData"]["subject"]
        if type(s)!=str:
            sub.extend(s)
        else:
            sub.append(s)
    except:
        pass
    try:
        d = res["hits"]["hits"][0]["_source"]["metaData"]["date"]
        if type(d)!=str:
            date.extend(d)
        else:
            date.append(d)
    except:
        pass
#     try:
#         m = res["hits"]["hits"][0]["_source"]["MD5"]
#         if type(m)!=str:
#             md5.extend(m)
#         else:
#             md5.append(m)
#     except:
#         pass
    lis.extend([[name,frm,to,sub,date]])
    if i==100:
        with open("entityExtrOA__.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)
        lis=[]
        i=0
if i!=0:
    with open("entityExtrOA__.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)

100%|██████████| 1602/1602 [08:50<00:00,  3.02it/s]


In [40]:
df = pd.read_csv("entityExtrOA__.csv")

In [41]:
null_lis = [val for val in df["email_date"] if len(eval(val))==0]
len(null_lis)

177

In [42]:
k = []
for i in range(len(df["email_date"])):
    if len(df["email_date"][i]) <= 4:
        k.append(i)

In [43]:
len(k)

177

In [44]:
import csv
header = ["md5"]
with open("OA_missing_md5_lis_2.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [45]:
for i in k:
    lis = []
    lis.append(df["md5"][i])
    with open("OA_missing_md5_lis_2.csv","a") as f:
                csvwriter = csv.writer(f)
                csvwriter.writerow(lis)

In [106]:
import pandas as pd
df1 = pd.read_excel("entityExtractionCD_sep_oct.xlsx")
df1.head(1)

,sku,product_description,cost_per_unit,vendor_name,vendor_number,date,purchase_agreement_date,cbm_number,cbm_name,contract_number,promotional_deal,signatures,format,source,filename,md5,program_name
0,439402,BISL SPINWAVE ROBOT,500,BITCO- BISSELL International Trading Company,2454,"August 25th, 2022",NaN,030 - John Hooke,NaN,NaN,All orders placed during the ordering window f...,0,old,email,BISSELL_-0439402_Q3_&_Q4_Cost_Down_signed.pdf,36de90459b3d13079a20c2557dc4eb3b,NaN


In [107]:
df2 = pd.read_csv("entityExtrCD.csv")
df2.head(1)

,md5,from,to,subject,email_date
0,36de90459b3d13079a20c2557dc4eb3b,"['""Wright', 'Steve.Wright@bissell.com']","['Meg.Casbourn@cantire.com', '""Chalmers', 'Nel...",['[EXTERNAL] - RE: 2022 ODS Q3 & Q4 BISSELL ru...,['2022/09/06 21:02:35']


In [107]:
df = pd.merge(df1, df2, on='filename', how='left')
df.head()

,vendor_number,vendor_name,cbm_number,cbm_name,contract_number,contract_date,amount,program_name,filename,md5,from,to,subject,email_date
0,9680,EURO-PRO CORPORATION,30.0,NaN,NaN,2022-09-12 00:00:00,200.06,Vendor Funds to Support Webshop Orders for DWO...,Shark_9680_-_Margin_O&A_Q1_Q2_-_Webshop_Orders...,dabd09db079b69c5e71cff3fd6ca995c,['Meg.Casbourn@cantire.com'],"['John.Hooke@cantire.com', 'REddy@sharkninja.c...",['[EXTERNAL] - RE: SHARK ALM Activity Tracking...,['2022/09/21 13:27:13']
1,3290,RUST-OLEUM CORP,NaN,NaN,NaN,NaN,Per Unit,2022 Q1 Cost Downs,22-0304-3290-163.pdf,965aebacf070da9ef75b1c45e0623023,['RForsyth@rustoleum.ca'],['Andrew.Holder@cantire.com'],['[EXTERNAL] - RE: 22-0304-3290-163.docx'],['2022/01/19 07:50:38']
2,NaN,NaN,58.0,NaN,NaN,NaN,Per Unit ($),2022 Q4 Cost Downs,22-0349-4109-058.pdf,0e0f9d79ac55cf5cdd790b69eaf9a947,"['""Houle', 'houle.t@pg.com']",['Andrew.Holder@cantire.com'],['[EXTERNAL] - RE: Deals 0349/350/351 Updated ...,['2022/09/14 19:55:02']
3,NaN,NaN,NaN,NaN,NaN,NaN,See Schedule A attached hereto,Deal 329/2022,Deal_329_Cost_Down_OA_Agreement_-_v9344.pdf,feacec6cb818338171c53fb5af32483e,['Shelley.Silliker@cantire.com'],['david.wright@advantagesolutions.net'],['[EXTERNAL] - cost downs Deal 329 and 330'],['2022/01/17 19:35:32']
4,4520,MTD PRODUCTS LTD.,90.0,NaN,NaN,August 29th 2022,14569,Non Advertising O&A,OverAbove_2022_MTDZT.pdf,48e1cf410ba92719d34f1b8e3f82ce03,['Karl.Solis@cantire.com'],['Vendor.Support@cantire.com'],['[EXTERNAL] - FW: O&A for ZT Tractors - V# 21...,['2022/09/26 15:59:52']


In [108]:
df.to_csv("oa-sep-oct-fin.csv")

In [109]:
pwd

'/mnt/cantire-analytics/datarepo/zz'

In [11]:
cd zz

/mnt/cantire-analytics/datarepo/zz


In [12]:
import pandas as pd
dff = pd.read_excel("Aug-Oct-OA_CD.xlsx",sheet_name="Aug-Oct-OA_CD")
print(dff.shape)
print(dff.head(1))

(2445, 5)
  classification              Filename  \
0             OA  22-0351-G48P-114.pdf   

                                            FilePath  \
0  cantire-2022-08/_doc/39f58eb48812784b6e38fcb37...   

                                MD5                        Email Path  
0  39f58eb48812784b6e38fcb3721834d4  fd46d4149eb7798b576ce097949eebe5  


In [14]:
len(dff["MD5"].unique())

2028

In [16]:
md5s = dff["MD5"].unique()

In [17]:
md5s[0]

'39f58eb48812784b6e38fcb3721834d4'

In [18]:
len(md5s)

2028

In [19]:
pwd

'/mnt/cantire-analytics/datarepo/zz'

In [32]:
import csv
header = ["md5","from","to","cc","subject","email_date"]
with open("aug_oct_oa-cd.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [33]:
from tqdm import tqdm
i=0
lis = []
# indexes = ["cantire-2021-08","cantire-2021-09","cantire-2021-10","cantire-2021-11","cantire-2021-12","cantire-2022-01","cantire-2022-02",
#           "cantire-2022-03","cantire-2022-04","cantire-2022-05","cantire-2022-06","cantire-2022-07","cantire-2022-08"]
indexes = ["cantire-2022-08","cantire-2022-09","cantire-2022-10"]
for name in tqdm(md5s):
    i+=1
    body = {
        "query":{
            "bool":{
                "must":[{
                    "match_phrase_prefix":{
                        "Attachment.MD5":name
                    }
                },
                {
                    "match_phrase_prefix":{
                        "fileType":"Email"
                    }
                }]
            }
        }
    }
    res = es.search(index=indexes,body = body)
    frm = []
    to = []
    cc = []
    sub = []
    date = []
    try:
        f = res["hits"]["hits"][0]["_source"]["metaData"]["from"]
        if type(f)!=str:
            frm.extend(f)
        else:
            frm.append(f)
    except:
        pass
    try:
        t = res["hits"]["hits"][0]["_source"]["metaData"]["to"]
        if type(t)!=str:
            to.extend(t)
        else:
            to.append(t)
    except:
        pass
    try:
        c = res["hits"]["hits"][0]["_source"]["metaData"]["cc"]
        if type(t)!=str:
            cc.extend(c)
        else:
            cc.append(c)
    except:
        pass
    try:
        s = res["hits"]["hits"][0]["_source"]["metaData"]["subject"]
        if type(s)!=str:
            sub.extend(s)
        else:
            sub.append(s)
    except:
        pass
    try:
        d = res["hits"]["hits"][0]["_source"]["metaData"]["date"]
        if type(d)!=str:
            date.extend(d)
        else:
            date.append(d)
    except:
        pass
    lis.extend([[name,frm,to,cc,sub,date]])
    if i==100:
        with open("aug_oct_oa-cd.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)
        lis=[]
        i=0
if i!=0:
    with open("aug_oct_oa-cd.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)

100%|██████████| 2028/2028 [08:29<00:00,  3.98it/s]


In [35]:
df = pd.read_csv("aug_oct_oa-cd.csv")
df.shape

(2028, 6)

In [36]:
len([val for val in df["from"] if len(eval(val))==0])

52

In [48]:
k = []
for i in range(len(df["from"])):
    if len(df["from"][i]) <= 4:
        k.append(i)
len(k)

52

In [53]:
k[13]

171

In [49]:
import csv
header = ["md5"]
with open("AUG_OCT_missing_md5_lis.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [54]:
for i in k:
    lis = []
    lis.append(df["md5"][i])
    with open("AUG_OCT_missing_md5_lis.csv","a") as f:
                csvwriter = csv.writer(f)
                csvwriter.writerow(lis)

In [37]:
import pandas as pd
df3 = pd.read_excel("Aug-Oct-OA_CD.xlsx",sheet_name="Aug-Oct-OA_CD")
df3.shape

(2445, 5)

In [38]:
df4 = pd.read_csv("aug_oct_oa-cd.csv")
df4.head(1)

,md5,from,to,cc,subject,email_date
0,39f58eb48812784b6e38fcb3721834d4,[],[],[],[],[]


In [39]:
df4.rename(columns = {'md5':'MD5'}, inplace = True)

In [41]:
df4.head(1)

,MD5,from,to,cc,subject,email_date
0,39f58eb48812784b6e38fcb3721834d4,[],[],[],[],[]


In [42]:
dff = pd.merge(df3, df4, on='MD5', how='left')

,classification,Filename,FilePath,MD5,Email Path,from,to,cc,subject,email_date
0,OA,22-0351-G48P-114.pdf,cantire-2022-08/_doc/39f58eb48812784b6e38fcb37...,39f58eb48812784b6e38fcb3721834d4,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
1,OA,22-0349-G48P-114.pdf,cantire-2022-08/_doc/531dfc97145f6ebbc18870b1b...,531dfc97145f6ebbc18870b1b7015d99,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
2,OA,22-0346-G48P-114.pdf,cantire-2022-08/_doc/d38a2de9a71ae4f3b6e359c16...,d38a2de9a71ae4f3b6e359c1603d5c72,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
3,OA,22-0344-G48P-114.pdf,cantire-2022-08/_doc/159c408b89896a00f3dc6c57d...,159c408b89896a00f3dc6c57d0c66bbe,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
4,OA,22-0348-G48P-114.pdf,cantire-2022-08/_doc/3b4c529c842bfd1fd48477ac3...,3b4c529c842bfd1fd48477ac3e1641a4,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]


In [43]:
dff.tail()

,classification,Filename,FilePath,MD5,Email Path,from,to,cc,subject,email_date
2440,OA,CTC_2022_Over_&_Above_Agreement_-_Poppa_Corn_C...,cantire-2022-10/_doc/f29e00c5bfb38a567f130aadc...,f29e00c5bfb38a567f130aadcad5fc83,d235b8c188ea5951af2fc5f5541f8e7e,['gregh@poppacorncorp.com'],['Shelley.Silliker@cantire.com'],['petiab@poppacorncorp.com'],['[EXTERNAL] - RE: 2022 Summer Markdown Agreem...,['2022/10/05 19:44:47']
2441,OA,1274_DNT8659216.pdf,cantire-2022-10/_doc/4de7ce02d5c5bf2e92dc092be...,4de7ce02d5c5bf2e92dc092bebd7c495,4117743db093caf859d6d445fe59b8ab,['APSupport@cantire.com'],"['CanAR@scj.com', 'BDimock@scj.com', 'nalsar@s...",['APSupport@cantire.com'],['[EXTERNAL] - 1274 CTC Debit/Credit Note Atta...,['2022/10/05 19:29:08']
2442,OA,1274_DNT8659216.pdf,cantire-2022-10/_doc/4de7ce02d5c5bf2e92dc092be...,4de7ce02d5c5bf2e92dc092bebd7c495,4117743db093caf859d6d445fe59b8ab,['APSupport@cantire.com'],"['CanAR@scj.com', 'BDimock@scj.com', 'nalsar@s...",['APSupport@cantire.com'],['[EXTERNAL] - 1274 CTC Debit/Credit Note Atta...,['2022/10/05 19:29:08']
2443,OA,CTC_2022_Over_&_Above_Agreement_-_Poppa_Corn_C...,cantire-2022-10/_doc/f29e00c5bfb38a567f130aadc...,f29e00c5bfb38a567f130aadcad5fc83,d235b8c188ea5951af2fc5f5541f8e7e,['gregh@poppacorncorp.com'],['Shelley.Silliker@cantire.com'],['petiab@poppacorncorp.com'],['[EXTERNAL] - RE: 2022 Summer Markdown Agreem...,['2022/10/05 19:44:47']
2444,OA,Andrea-Prima_OA_Contract_067_Q1_2022.pdf,cantire-2022-10/_doc/f09a6bbda3206359e8323a8fa...,f09a6bbda3206359e8323a8fa3a8639c,e52410aa1abf2d0b22f2b4ab669c0bf9,['Jiea.Jeong@cantire.com'],['Amanda.Lee@cantire.com'],"['Bonnie.Choi@cantire.com', 'Sarah.Lackey@cant...",['[EXTERNAL] - RE: Prima Q1-O&A Contract'],['2022/10/11 14:01:22']


In [44]:
dff.to_csv("AUG-OCT-OA-CD.csv")

In [45]:
d = pd.read_csv("AUG-OCT-OA-CD.csv")
d

,Unnamed: 0,classification,Filename,FilePath,MD5,Email Path,from,to,cc,subject,email_date
0,0,OA,22-0351-G48P-114.pdf,cantire-2022-08/_doc/39f58eb48812784b6e38fcb37...,39f58eb48812784b6e38fcb3721834d4,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
1,1,OA,22-0349-G48P-114.pdf,cantire-2022-08/_doc/531dfc97145f6ebbc18870b1b...,531dfc97145f6ebbc18870b1b7015d99,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
2,2,OA,22-0346-G48P-114.pdf,cantire-2022-08/_doc/d38a2de9a71ae4f3b6e359c16...,d38a2de9a71ae4f3b6e359c1603d5c72,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
3,3,OA,22-0344-G48P-114.pdf,cantire-2022-08/_doc/159c408b89896a00f3dc6c57d...,159c408b89896a00f3dc6c57d0c66bbe,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
4,4,OA,22-0348-G48P-114.pdf,cantire-2022-08/_doc/3b4c529c842bfd1fd48477ac3...,3b4c529c842bfd1fd48477ac3e1641a4,fd46d4149eb7798b576ce097949eebe5,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
2440,2440,OA,CTC_2022_Over_&_Above_Agreement_-_Poppa_Corn_C...,cantire-2022-10/_doc/f29e00c5bfb38a567f130aadc...,f29e00c5bfb38a567f130aadcad5fc83,d235b8c188ea5951af2fc5f5541f8e7e,['gregh@poppacorncorp.com'],['Shelley.Silliker@cantire.com'],['petiab@poppacorncorp.com'],['[EXTERNAL] - RE: 2022 Summer Markdown Agreem...,['2022/10/05 19:44:47']
2441,2441,OA,1274_DNT8659216.pdf,cantire-2022-10/_doc/4de7ce02d5c5bf2e92dc092be...,4de7ce02d5c5bf2e92dc092bebd7c495,4117743db093caf859d6d445fe59b8ab,['APSupport@cantire.com'],"['CanAR@scj.com', 'BDimock@scj.com', 'nalsar@s...",['APSupport@cantire.com'],['[EXTERNAL] - 1274 CTC Debit/Credit Note Atta...,['2022/10/05 19:29:08']
2442,2442,OA,1274_DNT8659216.pdf,cantire-2022-10/_doc/4de7ce02d5c5bf2e92dc092be...,4de7ce02d5c5bf2e92dc092bebd7c495,4117743db093caf859d6d445fe59b8ab,['APSupport@cantire.com'],"['CanAR@scj.com', 'BDimock@scj.com', 'nalsar@s...",['APSupport@cantire.com'],['[EXTERNAL] - 1274 CTC Debit/Credit Note Atta...,['2022/10/05 19:29:08']
2443,2443,OA,CTC_2022_Over_&_Above_Agreement_-_Poppa_Corn_C...,cantire-2022-10/_doc/f29e00c5bfb38a567f130aadc...,f29e00c5bfb38a567f130aadcad5fc83,d235b8c188ea5951af2fc5f5541f8e7e,['gregh@poppacorncorp.com'],['Shelley.Silliker@cantire.com'],['petiab@poppacorncorp.com'],['[EXTERNAL] - RE: 2022 Summer Markdown Agreem...,['2022/10/05 19:44:47']
